In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 46.8 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 64.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=23cdae8d15f4b9c1d625010ee96a32b4a1a364af2e736446dda89e009b7b96f9
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.

In [5]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [6]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [7]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [8]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [9]:
filter_phrase = df['tags'] == "phrase"
df_filtered = df[filter_phrase]
# print (df_filtered)

In [10]:
filter_phrase_validation = df_validation['tags'] == "phrase"
df_filtered_validation = df_validation[filter_phrase_validation]
# print (df_filtered_validation)

In [11]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [12]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [ ]:
# wordLemm = WordNetLemmatizer()
# def processed_paragraph(paragraph):
#     # Lower Casing
#     paragraph = paragraph.lower()
#     # Remove punctuations
#     paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
#     # Tokenizing words
#     tokens = word_tokenize(paragraph)
#     # Removing Stop Words
#     final_tokens = [w for w in tokens if w not in stopword]
#     # Reducing a word to its word stem using lemmatization
#     finalwords=[]
#     for w in final_tokens:
#       if len(w)>1:
#         word = wordLemm.lemmatize(w)
#         finalwords.append(word)
#     return ' '.join(finalwords)

In [13]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [14]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
new_df = generate_new_df(df_filtered)

In [17]:
new_df_validation = generate_new_df(df_filtered_validation)

In [18]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [19]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [20]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# questions= new_df['postText']
# answers= new_df['spoiler']
# stories_not_summarized = new_df['paragraph']
# tags = new_df['tags']

# print(stories_not_summarized[0])




In [21]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=34d9fcc8bd41e170bf3e77ed6ab665ee8d21e761a34c2866a9dc8d60e8d17e89
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [22]:
import torch

In [ ]:
# from transformers import pipeline
# import requests
# import pprint
# import time
# pp = pprint.PrettyPrinter(indent=14)

# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')


# # model = T5ForConditionalGeneration.from_pretrained('t5-small')
# # tokenizer = T5Tokenizer.from_pretrained('t5-small')
# device_cuda = torch.device('cuda')

In [23]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [24]:
from datetime import datetime

In [25]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [26]:
# # summarize with T5
# # summarizer_t5 = pipeline(task='summarization', model="t5-large")
# summarizer_t5 = pipeline(task='summarization', model="t5-small")

In [ ]:
#FINE TUNING START

In [27]:
!pip install transformers #==2.9.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00


In [28]:
from datasets import Dataset
from datasets import dataset_dict

In [29]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
import torch
import numpy as np
import pandas as pd
from transformers import (TrainingArguments, Trainer, EarlyStoppingCallback)
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [52]:
# max_input = 512
# max_target = 128
# batch_size = 3
model_checkpoints = "t5-small"

In [53]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [31]:
input_text = "Sumarize:" + "condition: " + new_df['postText'] + " paragraph: " + new_df["paragraph"]
target_text = new_df["spoiler"]


data_to_process_train = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [32]:
data_to_process_train

,input_ids,labels
1,Sumarize:condition: nasa sets date for full re...,2070
2,Sumarize:condition: this is what makes employe...,intellectual stimulation
4,Sumarize:condition: the perfect way to cook ri...,in a rice cooker
10,Sumarize:condition: analysis this may be the m...,750 percent
11,Sumarize:condition: teenmom2 star pbandjenelle...,boy
...,...,...
3178,Sumarize:condition: he dug a huge hole in his ...,own underground bunker!
3179,Sumarize:condition: best buy has an insane xbo...,$50 off
3183,Sumarize:condition: student forced to carry pa...,student at Hampton University in Virginia
3198,Sumarize:condition: you need to see this twitt...,@beyoncefan666


In [33]:
df1_dict = data_to_process_train.to_dict(orient = 'list')

In [34]:
dict_train_ = Dataset.from_dict(df1_dict)

In [35]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1367
})

In [36]:
input_text = "Sumarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_validation = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [37]:
data_to_process_train

,input_ids,labels
1,Sumarize:condition: nasa sets date for full re...,2070
2,Sumarize:condition: this is what makes employe...,intellectual stimulation
4,Sumarize:condition: the perfect way to cook ri...,in a rice cooker
10,Sumarize:condition: analysis this may be the m...,750 percent
11,Sumarize:condition: teenmom2 star pbandjenelle...,boy
...,...,...
3178,Sumarize:condition: he dug a huge hole in his ...,own underground bunker!
3179,Sumarize:condition: best buy has an insane xbo...,$50 off
3183,Sumarize:condition: student forced to carry pa...,student at Hampton University in Virginia
3198,Sumarize:condition: you need to see this twitt...,@beyoncefan666


In [38]:
df2_dict = data_to_process_validation.to_dict(orient = 'list')

In [39]:
dict_validation_ = Dataset.from_dict(df2_dict)

In [40]:
dict_validation_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 335
})

In [41]:
input_text = "Sumarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_test = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [42]:
data_to_process_test

,input_ids,labels
2,Sumarize:condition: here ’ s how much you shou...,20%
5,Sumarize:condition: this popular soda could cu...,Sprite
6,Sumarize:condition: the anytime snack you wont...,Smoky Paprika-Baked Garbanzo Beans
11,Sumarize:condition: guess who obama just dined...,Anthony Bourdain
15,Sumarize:condition: what are mosquitocontrol w...,Dibrom
...,...,...
786,Sumarize:condition: kfc ’ s secret ingredient ...,white pepper
787,Sumarize:condition: sex experts think that mor...,anal sex
791,Sumarize:condition: both campaigns stunned aft...,Hillary Clinton
796,Sumarize:condition: this texas gop elector ann...,Christopher Suprun


In [43]:
df3_dict = data_to_process_test.to_dict(orient = 'list')

In [44]:
dict_test_ = Dataset.from_dict(df3_dict)

In [45]:
dict_test_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 335
})

In [ ]:
# df1_dict = data_to_process_train.to_dict(orient = 'list')
# df2_dict = data_to_process_validation.to_dict(orient = 'list')
# df3_dict = data_to_process_test.to_dict(orient = 'list')


# dict_train_ = Dataset.from_dict(df1_dict)
# dict_validation_ = Dataset.from_dict(df2_dict)
# dict_test_ = Dataset.from_dict(df3_dict)

In [46]:
dict_all_2 = dataset_dict.DatasetDict({"train":dict_train_, "validation":dict_validation_, "test": dict_test_})

In [47]:
dict_all_2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1367
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 335
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 335
    })
})

In [48]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1367
})

In [49]:
model = "t5-small"

In [54]:
tokenizer1 = tokenizer.from_pretrained("t5-small")

In [55]:
def preprocess_data(example):
    input_text = example['input_ids']
    target_text = example['labels']

    input_ids = tokenizer1(input_text, padding="max_length", truncation=True, max_length= 1000).input_ids

    with tokenizer1.as_target_tokenizer():
      target_ids = tokenizer1(target_text, padding='max_length', truncation=True, max_length= 500).input_ids
    # target_ids = tokenizer(target_text, padding="max_length", truncation=True, max_length= 500, return_tensors="pt").input_ids
    
    return {"input_ids": input_ids, "labels": target_ids}
    # return {"input_text": input_text, "target_text": target_text}

In [56]:
train_dataset = dict_train_
test_dataset = dict_validation_

train_dataset_processed = train_dataset.map(preprocess_data)
test_dataset_processed = test_dataset.map(preprocess_data)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [57]:
train_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1367
})

In [58]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 335
})

In [59]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [60]:
model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels = 4)

In [61]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.3 MB/s eta 0:00:00


In [62]:
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
bertscore_metric = load_metric("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [63]:
#out_1 = []
#out_2 = []

def compute_metrics(eval_pred):
    logits_full, labels = eval_pred

    #out_1.append(logits_full[0])
    #out_2.append(logits_full[1])

    predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_bleu = [pred.split() for pred in decoded_preds]
    decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
    bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
    meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
    return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [64]:
import shutil


In [67]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/T5_finetuning_phrase_results_conditional_all_sizelimited",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    # save_steps=0,
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu",
    # greater_is_better=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 3)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)


In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.031900,0.034537
2,0.028900,0.029659
3,0.025900,0.025021
4,0.023800,0.023164
5,0.023100,0.022151
6,0.022300,0.021743
7,0.020700,0.021087
8,0.020900,0.020714
9,0.020900,0.020510
10,0.019500,0.020323


TrainOutput(global_step=8550, training_loss=0.019884965015433687, metrics={'train_runtime': 3212.5592, 'train_samples_per_second': 10.638, 'train_steps_per_second': 2.661, 'total_flos': 9033800908800000.0, 'train_loss': 0.019884965015433687, 'epoch': 25.0})

In [ ]:
# result = model.

In [69]:
metrics = trainer.evaluate()


In [70]:
metrics

{'eval_loss': 0.019491121172904968,
 'eval_runtime': 10.5581,
 'eval_samples_per_second': 31.729,
 'eval_steps_per_second': 7.956,
 'epoch': 25.0}

In [71]:
best_model_checkpoint = trainer.state.best_model_checkpoint
print(f"Best model checkpoint: {best_model_checkpoint}")

Best model checkpoint: None


In [ ]:
dict_validation_["labels"]

In [73]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["labels"],
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [74]:
# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summaries_after_tuning_all = []

def generate_batch(lst, batch_size):
    """  Yields batch of specified size """
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]
        # summaries_after_tuning = generate_summary(test_samples, model)[1]
        # summaries_after_tuning_all.append(summaries_after_tuning)


# test_samples = dict_validation_.select(range())
test_samples = generate_batch(dict_validation_, 16)

# # summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

for batch in test_samples:
  # print(batch)
  summaries_after_tuning = generate_summary(batch, model)[1]
  summaries_after_tuning_all.append(summaries_after_tuning)

In [75]:
summaries_after_tuning_all

[['20%',
  'Sprite',
  'Garbanzo Beans Smoky Paprika-Baked',
  'Anthony Bourdain',
  'Dibrom',
  'They don’t fart',
  'beginnt später',
  'iPad Pros',
  'Stace Nelson',
  '         ',
  'Edward Gorey',
  'Rag & Bone',
  '',
  'Southern Fly squirrel',
  '',
  '6 August'],
 ['durchschnittlich',
  '"aa"',
  'Arizona Republic',
  'apokalyptic omen',
  'Rapamycin',
  'Santa Clara University',
  'Oklahoma Wesleyan University',
  'Edan Lepucki',
  'Tim Masthay',
  "World's Largest Pumpkin",
  '$25K',
  'University of Tennessee',
  'Crazy Frog',
  '',
  'Amazon',
  'Kuru'],
 ['Dieffenbachia',
  'Baby food',
  'Michael and Kirk',
  '1,4-dioxane',
  'Bak Bakgut',
  'Wyoming',
  'Nvidia',
  '2,1 coffee drinks',
  'Gigie Stück Papier',
  'neutralität net',
  'Fall of the Resistance.',
  '4,47 milliards',
  'Schweiz',
  '2 million',
  'Schuh',
  'Gov. Rick Snyder (R-Michigan)'],
 ['',
  '',
  'Arsenic, Arsenic, Arsenic, Arsenic',
  '$900 million',
  'EasyAuto123.com',
  'Guyana',
  'Gilbert Arenas'

'Einige der plot elements are so disturbing that they make him feel sick.'

In [76]:
stories_list =[]
for i in range(len(summaries_after_tuning_all)):
  for j in range(len(summaries_after_tuning_all[i])):
    summary=summaries_after_tuning_all[i][j]
    stories_list.append(summary)


print(stories_list)

['20%', 'Sprite', 'Garbanzo Beans Smoky Paprika-Baked', 'Anthony Bourdain', 'Dibrom', 'They don’t fart', 'beginnt später', 'iPad Pros', 'Stace Nelson', '         ', 'Edward Gorey', 'Rag & Bone', '', 'Southern Fly squirrel', '', '6 August', 'durchschnittlich', '"aa"', 'Arizona Republic', 'apokalyptic omen', 'Rapamycin', 'Santa Clara University', 'Oklahoma Wesleyan University', 'Edan Lepucki', 'Tim Masthay', "World's Largest Pumpkin", '$25K', 'University of Tennessee', 'Crazy Frog', '', 'Amazon', 'Kuru', 'Dieffenbachia', 'Baby food', 'Michael and Kirk', '1,4-dioxane', 'Bak Bakgut', 'Wyoming', 'Nvidia', '2,1 coffee drinks', 'Gigie Stück Papier', 'neutralität net', 'Fall of the Resistance.', '4,47 milliards', 'Schweiz', '2 million', 'Schuh', 'Gov. Rick Snyder (R-Michigan)', '', '', 'Arsenic, Arsenic, Arsenic, Arsenic', '$900 million', 'EasyAuto123.com', 'Guyana', 'Gilbert Arenas', 'Alex Castellanos', 'Bringing the work to them', '"Gravity"', 'Wisconsin', 'Isla Fisher', '', '170', 'Atlanta'

In [77]:
# type(new_df['spoiler'][0:40])

In [78]:
type(stories_list[0:40])

list

In [79]:
stories_list_series = pd.Series((v for v in stories_list))

In [80]:
import pandas as pd

In [94]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 335
})

In [97]:
dataframe = pd.DataFrame({ 'postText': new_df_validation["postText"], 'actual_spoiler':test_dataset['labels'], 'gen_spoiler':stories_list })

In [98]:
dataframe

,postText,actual_spoiler,gen_spoiler
2,here ’ s how much you should be tipping your h...,20%,20%
5,this popular soda could cure your hangovers sc...,Sprite,Sprite
6,the anytime snack you wont feel guilty about e...,Smoky Paprika-Baked Garbanzo Beans,Garbanzo Beans Smoky Paprika-Baked
11,guess who obama just dined with in vietnam,Anthony Bourdain,Anthony Bourdain
15,what are mosquitocontrol workers spraying in m...,Dibrom,Dibrom
...,...,...,...
786,kfc ’ s secret ingredient is the most underapp...,white pepper,
787,sex experts think that more straight men are e...,anal sex,Anal sex
791,both campaigns stunned after houston chronicle...,Hillary Clinton,Hillary Clinton
796,this texas gop elector announces that he wont ...,Christopher Suprun,Christopher Suprun


In [106]:
file = open("T5_conditional_finetuning_no_tags_phrase_results.txt", "w")

#convert variable to string
# str_1 = repr(dataframe)
str2 = dataframe.to_json()
file.write("T5_conditional_finetuning_no_tags_phrase_results = " + str2 + "\n")

#close file
file.close()

In [83]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/loss,█▆▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▁▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂█
eval/samples_per_second,██▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▁
eval/steps_per_second,██▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▁
train/epoch,▁▁▁▂▂▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
#EVALUATION METRICS

In [107]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [108]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [109]:
bleu_scores

[1.2213386697554703e-77,
 1.0,
 0.9279497539902547,
 1.0,
 1.0,
 1.0,
 3.1923134147786836e-78,
 0.45046627229833414,
 1.0,
 1.1070900996592687e-231,
 1.0,
 1.0,
 0,
 0.6425503166524515,
 0,
 0.5555238068023582,
 5.1945147945938015e-155,
 1.4488496539373276e-231,
 0.7828785637123031,
 0.8153551038173115,
 0.8633400213704505,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 1.0,
 8.636168555094496e-78,
 1.0,
 0.8633400213704505,
 1.0,
 1.0,
 3.771272693679486e-78,
 1.0,
 1.0,
 0.8843946454355334,
 3.062358125062913e-78,
 0.7644926165660443,
 1.0,
 0.1832567180568652,
 1.4959003140449574e-231,
 0.2698809072033866,
 1.0244914152188952e-231,
 1.0,
 0,
 0,
 0.13793309405261964,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8298325792963526,
 1.0,
 1.0,
 1.0,
 0,
 1.2213386697554703e-77,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 1.0,
 0.6721651668262854,
 0.8277932960330121,
 0.6816650778781156,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.2213386697554703e-77,
 1.0,
 0.14700587655767763,
 0,
 1.0,
 1.0,
 5.395638

In [110]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.64

In [111]:
bleu_min = min(bleu_scores)
bleu_min

0

In [112]:
bleu_max = max(bleu_scores)
bleu_max

1.0

In [118]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.9025139799587886

In [119]:
import sys
from nltk.translate.meteor_score import meteor_score

In [120]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [121]:
meteor_scores

[0.9375,
 0.5,
 0.9375,
 0.9375,
 0.5,
 0.996,
 0.0,
 0.8928571428571429,
 0.9375,
 0.0,
 0.9375,
 0.9814814814814815,
 0.0,
 0.9814814814814815,
 0.0,
 0.25,
 0.0,
 0.3333333333333333,
 0.8928571428571429,
 0.25,
 0.5,
 0.9814814814814815,
 0.9814814814814815,
 0.9375,
 0.9375,
 0.9921875,
 0.9375,
 0.9814814814814815,
 0.9375,
 0.0,
 0.5,
 0.5,
 0.5,
 0.9375,
 0.9814814814814815,
 0.5,
 0.0,
 0.5,
 0.5,
 0.625,
 0.0,
 0.25,
 0.996,
 0.0,
 0.0,
 0.8152173913043478,
 0.0,
 0.9985422740524781,
 0.0,
 0.0,
 0.07575757575757576,
 0.9814814814814815,
 0.5,
 0.5,
 0.9375,
 0.9375,
 0.996,
 0.9814814814814815,
 0.5,
 0.9375,
 0.0,
 0.5,
 0.5,
 0.5,
 0.5,
 0.9375,
 0.9814814814814815,
 0.0,
 0.9375,
 0.8066666666666668,
 0.981329690346084,
 0.625,
 0.9921875,
 0.9375,
 0.9814814814814815,
 0.9375,
 0.9814814814814815,
 0.5,
 0.5,
 0.5,
 0.08771929824561403,
 0.0,
 0.9375,
 0.9375,
 0.0,
 0.5,
 0.9990234375,
 0.0,
 0.5,
 0.0,
 0.25,
 0.9375,
 0.8412698412698414,
 0.9375,
 0.5,
 0.7500000000000

In [122]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.55

In [123]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.9990234375

In [124]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [125]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.5

In [ ]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [ ]:
bert_scores_P = []
bert_scores_R = []
bert_scores_F1 = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  P, R, F1 = score(gen_spoiler_strip, actual_spoiler_strip, lang="en", verbose=True)
  bert_scores_P.append(P)
  bert_scores_R.append(R)
  bert_scores_F1.append(F1)

  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  # meteor_scores.append(meteor)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:13                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/bert_score/score.py:70 in score                          │
│                                                                                                  │
│    67 │   │   │   │     multiple references, the returned score of this candidate is             │
│    68 │   │   │   │     the *best* score among all references.                                   │
│    69 │   """                                                                                    │
│ ❱  70 │   assert len(cands) == len(refs), "Different number of candidates and references"        │
│    71 │                                                                                          │
│    72 │   assert (                                                                               │
│    73 │   │   lang is not None or model_type is not None                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: Different number of candidates and references

In [ ]:
# P, R, F1 = score(cands, refs, lang="en", verbose=True)

In [ ]:
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  print("\n")
  print(ref)
  print("\n")
  print(hypo)
  print("\n")



['tom', 'brady', "'", 's', 'definition', 'of', 'catching', 'up', 'involves', 'much', 'more', 'than', 'just', 'a', 'meal', '.', 'wes', 'welker', 'contacted', 'the', 'patriots', 'quarterback', 'friday', 'morning', 'on', 'a', 'miami', 'radio', 'station', '.', 'bra']


['how', 'about', 'that', 'morning', 'we', 'go', 'throw', '?']




['nasa', 'scientists', 'are', 'predicting', 'the', 'hole', 'in', 'the']


['2070']




['research', 'by', 'hiring', 'software', 'provider', 'cangrade', 'found', 'that', 'being', 'intellectually', 'stimulated']


['intellectual', 'stimulation']




['cal', 'newport', "'", 's', 'investigation', 'into', 'how', 'passionate', 'people', 'like', 'steve', 'jobs', 'really', 'got', 'started', 'as', 'well', 'as', 'what', 'scientists', 'say', 'predicts', 'happiness', 'and', 'fuels', 'great', 'accomplishment', 'newport', '.', 'ryan', 'holiday', 'author', 'of', 'ego', 'is', 'the', 'enemy', 'your', 'passion', 'may', 'be', 'the', 'very', 'enemy', '.', 'the', 'only', 'way', 

In [ ]:
bert_scores_P

In [ ]:
bert_scores_R

In [ ]:
bert_scores_F1